# Convert h5 model to trt model

In [1]:
import sys 
import os

NOMEROFF_NET_DIR = "../../"

# setup result model dirs
SAVED_MODEL_DIR  = os.path.join(NOMEROFF_NET_DIR, "./models/saved_model_ocr_eu")
TRT_SAVED_MODEL_DIR  = os.path.join(NOMEROFF_NET_DIR, "./models/trt_saved_model_ocr_eu")

sys.path.append(NOMEROFF_NET_DIR)

In [2]:
import tensorflow as tf
import matplotlib.image as mpimg

from NomeroffNet import  TextDetector

## load latest h5 model

In [3]:
textDetector = TextDetector.get_static_module("eu")()
textDetector.load("latest")

In [4]:
textDetector.MODEL.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input_eu (InputLayer)       [(None, 128, 64, 1)] 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 64, 16)  160         the_input_eu[0][0]               
__________________________________________________________________________________________________
max1 (MaxPooling2D)             (None, 64, 32, 16)   0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 64, 32, 16)   2320        max1[0][0]                       
_______________________________________________________________________________________

## convert to pb model

In [5]:
tf.saved_model.save(textDetector.MODEL, SAVED_MODEL_DIR)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ../.././models/saved_model_ocr_eu/assets


## convert pb model to trt model

In [6]:
params = tf.experimental.tensorrt.ConversionParams(
    minimum_segment_size=50,
    max_workspace_size_bytes=1 << 25,
    precision_mode  = "FP16",
)

converter = tf.experimental.tensorrt.Converter(
    input_saved_model_dir=SAVED_MODEL_DIR,
    conversion_params=params)
converter.convert()
converter.save(TRT_SAVED_MODEL_DIR)

INFO:tensorflow:Linked TensorRT version: (7, 1, 3)
INFO:tensorflow:Loaded TensorRT version: (7, 1, 3)
INFO:tensorflow:Assets written to: ../.././models/trt_saved_model_ocr_eu/assets


## run trt model

In [7]:
saved_model_loaded = tf.saved_model.load(TRT_SAVED_MODEL_DIR)

In [8]:
import matplotlib.image as mpimg
import numpy as np

img_path = '../../examples/crop_np_images/JJF509.png'
imgs = [mpimg.imread(img_path)]
Xs = []
for img in imgs:
    Xs.append(textDetector.normalize_pb(img[:,:,:3]))

In [9]:
f = saved_model_loaded.signatures["serving_default"]

In [10]:
tensorX = tf.convert_to_tensor(np.array(Xs).astype(np.float32))

In [11]:
f(tensorX)["softmax_eu"]

<tf.Tensor: shape=(1, 32, 37), dtype=float32, numpy=
array([[[6.6797278e-05, 3.7861180e-03, 3.2796801e-05, ...,
         1.5257907e-02, 4.9162541e-06, 5.6130207e-01],
        [5.4070333e-12, 2.1345683e-09, 2.8306021e-12, ...,
         1.4669770e-08, 6.8138447e-13, 9.9999869e-01],
        [7.3350289e-21, 6.7008826e-18, 6.6396486e-22, ...,
         1.1695365e-17, 1.2215078e-20, 1.0000000e+00],
        ...,
        [5.8439135e-21, 4.0507810e-24, 4.5943956e-23, ...,
         4.2304713e-24, 2.1328006e-28, 1.0000000e+00],
        [1.3270533e-11, 5.1692188e-14, 6.1852731e-13, ...,
         1.0263432e-16, 8.0694207e-18, 9.9990261e-01],
        [7.7257534e-10, 3.9994804e-12, 2.8472367e-09, ...,
         4.2879600e-16, 2.1247113e-16, 1.4693139e-11]]], dtype=float32)>